# *Analiza knjig*
### Seznam knjig bom analiziral glede na različne parametre: ocena, kategorija, cena.

In [1]:
import pandas as pd
import plotly.graph_objs as go

knjige = pd.read_csv('obdelani-podatki/knjige_c.csv', index_col='id')
pd.options.display.max_rows = 15

### *Ocena ~ cena*
Prvo bom knjige razdelil v skupine po ocenah, in preveril ali imajo bolje ocenjene knjige v povprečju dražjo ceno. 

In [2]:
knjige_po_ocenah = knjige.groupby('ocena')
round(knjige_po_ocenah.cena.mean(), 2)

ocena
1    38.36
2    38.64
3    38.51
4    40.06
5    39.27
Name: cena, dtype: float64

### *Ocena ~ število knjig*
Preveril bom koliko knjig je v posamznem razredu(po ocenah).

In [3]:
knjige_po_ocenah.size()

ocena
1    226
2    196
3    203
4    179
5    196
dtype: int64

Opazimo, da so najcenejše knjige z najslabšo oceno, kar je pričakovano. Najdražje pa so v povprečju tiste z oceno 4.
Prav tako vidimo da je največ knjig slabo ocenjenih(z 1), najmanj pa tistih z oceno 4. <br>
Tu sem odkril povezavo: Več kot je bilo knjig v določeni skupini, nižja je bila povprečna cena. Če jih je bilo manj, je bila pa višja.

### *Kategorija ~ število knjig*
Prvo bom preučil število knjig v določeni kategoriji, da bom za nadaljno analizo lahko izločil kategorije z manj kot 'n' knjig, za primernejše rezultate.
<br>
Uporabil sem kodo s strani: https://www.w3schools.com/python/python_howto_remove_duplicates.asp
<br>
#### Graf ne prikaže vseh vrednosti če ni dovolj prostora, lahko uporabite opcijo zoom in približate na željene parametre!
#### Analiza vsebuje kategorije, ki so videti nesmiselne npr. 'Add a comment', 'Default'... Teh kategorij nisem izključil, ker je bistvo naloge analizirati podatke, če tudi niso nujno smiselni.

In [4]:
knjige_po_kategorijah = knjige.groupby('kategorija')
knjige_po_kategorijah.size()
st_knj = knjige_po_kategorijah.size().tolist()

sez_kat = knjige.kategorija.tolist()
sez_kat = list(dict.fromkeys(sez_kat))
sez_kat.sort()

trace1 = go.Bar(
    x = sez_kat,
    y = st_knj,
)
fig_1 = go.Figure(
    data = [trace1],
    layout_title_text = "Število knjig na kategorijo"
)
# Izriše graf števila knjig/vseh kategorij
fig_1

In [5]:
MIN_KNJ = 3
slovar = {sez_kat[i]: st_knj[i] for i in range(len(sez_kat))}
nov_slovar = dict([(value, key) for key, value in slovar.items()])

# Odstrani kategorije z manj kot MIN_KNJ knjigami
for key in list(nov_slovar):
    if key <= MIN_KNJ:
        del nov_slovar[key]

dict_ana = dict([(value, key) for key, value in nov_slovar.items()])

# Graf s kategorijami, ki jih bomo upoštevali, ker imajo zadostno število knjig.
trace1 = go.Bar(
    x = list(dict_ana.keys()),
    y = list(dict_ana.values()),
)
fig_2 = go.Figure(
    data = [trace1],
    layout_title_text = "Število knjig na kategorijo (z dovolj knjigami)"
)
# Izriše graf št. knjig/sprejemljivih kategorij
fig_2

### Kategorija ~ (o)cena

In [6]:
df_kat = round(knjige_po_kategorijah.mean(), 2)
df_kat['število_knjig'] = knjige_po_kategorijah.size()
df_kat = df_kat[df_kat.število_knjig > 3]
# V primeru da imata dve kategoriji enako povprečno oceno je boljša, tista ki ima več knjig
df_kat.sort_values(['ocena', 'število_knjig'], ascending=[False, False])

,ocena,cena,število_knjig
kategorija,,,
Christian Fiction,4.17,38.17,6
Health,3.75,57.12,4
Art,3.62,42.76,8
Poetry,3.53,39.93,19
Humor,3.40,37.19,10
...,...,...,...
Classics,2.47,40.57,19
Philosophy,2.36,37.25,11
Science Fiction,2.25,37.52,16


Opazimo da so najbolje ocenjenje kategorije: 'Christian fiction', 'Health', 'Art'... Najslabše pa 'Psychology', 'Biography', 'Science Fiction'...
<br>
Tu se mi zdi opazno da so podatki zgolj 'vzorčni', ker se mi ne zdijo smiselne porazdelitve.

In [7]:
# V primeru da imata dve kategoriji enako povprečno ceno je boljša, tista ki ima več knjig
df_kat.sort_values(['cena', 'število_knjig'], ascending=[False, False])

,ocena,cena,število_knjig
kategorija,,,
Health,3.75,57.12,4
New Adult,3.17,51.49,6
Sports and Games,3.00,45.69,5
Self Help,2.60,45.09,5
Travel,2.73,44.17,11
...,...,...,...
Religion,3.14,36.15,7
Business,2.92,36.03,12
Mystery,2.94,35.21,32


Vidimo da so najdražje knjige o zdravju, najcenejše pa o hrani in pijači.
<br>
Glede na to da je le ena izmed najbolje ocenjenih kategorij tudi med najdražjimi, ne moremo sklepati da obstaja povezava med oceno in povrpečno ceno kategorije.

In [8]:
# Graf na katerem so kategorije v odvisnosti od ocene, števila knjig in cene
cena = go.Bar(
    x = df_kat.index.tolist(),
    y = df_kat.cena.tolist(),
    name = 'cena'
)
ocena = go.Bar(
    x = df_kat.index.tolist(),
    y = df_kat.ocena.tolist(),
    name = 'ocena'
)
st_k = go.Bar(
    x = df_kat.index.tolist(),
    y = df_kat.število_knjig.tolist(),
    name = 'število knjig'
)

fig_3 = go.Figure(
    data = [cena, ocena, st_k],
    layout_title_text = "Kategorije"
)
fig_3

Iz grafa in prejšnjih dveh tabelaričnih prikazov razberemo da ne obstajajo fiksne povezave med številom knjig ter povprečno ceno in oceno na kategorijo.